# Installation
Just pip install:

```pip install omegaconf```

If you want to try this notebook after checking out the repository be sure to run 
```python setup.py develop``` at the repository root before running this code.

# Creating OmegaConf objects
### Empty

In [1]:
from omegaconf import OmegaConf
conf = OmegaConf.create()
print(conf)

{}


### From a dictionary

In [2]:
conf = OmegaConf.create(dict(k='v',list=[1,dict(a='1',b='2')]))
print(OmegaConf.to_yaml(conf))

k: v
list:
- 1
- a: '1'
  b: '2'



### From a list

In [3]:
conf = OmegaConf.create([1, dict(a=10, b=dict(a=10))])
print(OmegaConf.to_yaml(conf))

- 1
- a: 10
  b:
    a: 10



### From a yaml file

In [4]:
conf = OmegaConf.load('../source/example.yaml')
print(OmegaConf.to_yaml(conf))

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2



### From a yaml string

In [5]:
yaml = """
a: b
b: c
list:
- item1
- item2
"""
conf = OmegaConf.create(yaml)
print(OmegaConf.to_yaml(conf))

a: b
b: c
list:
- item1
- item2



### From a dot-list

In [6]:
dot_list = ["a.aa.aaa=1", "a.aa.bbb=2", "a.bb.aaa=3", "a.bb.bbb=4"]
conf = OmegaConf.from_dotlist(dot_list)
print(OmegaConf.to_yaml(conf))

a:
  aa:
    aaa: 1
    bbb: 2
  bb:
    aaa: 3
    bbb: 4



### From command line arguments

To parse the content of sys.arg:

In [7]:
# Simulating command line arguments
import sys
sys.argv = ['your-program.py', 'server.port=82', 'log.file=log2.txt']
conf = OmegaConf.from_cli()
print(OmegaConf.to_yaml(conf))

server:
  port: 82
log:
  file: log2.txt



# Access and manipulation
Input yaml file:

In [8]:
conf = OmegaConf.load('../source/example.yaml')
print(OmegaConf.to_yaml(conf))

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2



#### Object style access:

In [9]:
conf.server.port

80

#### dictionary style access

In [10]:
conf['log']['rotation']

3600

#### items in list

In [11]:
conf.users[0]

'user1'

#### Changing existing keys

In [12]:
conf.server.port = 81

#### Adding new keys

In [13]:
conf.server.hostname = "localhost"

#### Adding a new dictionary

In [14]:
conf.database = {'hostname': 'database01', 'port': 3306}

#### providing default values

In [15]:
conf.get('missing_key', 'a default value')

'a default value'

#### Accessing mandatory values
Accessing fields with the value *???* will cause a MissingMandatoryValue exception.
Use this to indicate that the value must be set before accessing.

In [16]:
from omegaconf import MissingMandatoryValue
try:
    conf.log.file
except MissingMandatoryValue as exc:
    print(exc)

Missing mandatory value: log.file
    full_key: log.file
    object_type=dict


# Variable interpolation

OmegaConf support variable interpolation, Interpolations are evaluated lazily on access.

## Config node interpolation

The interpolated variable can be the dot-path to another node in the configuration, and in that case the value will be the value of that node.

Interpolations are absolute by default. Relative interpolation are prefixed by one or more dots: The first dot denotes the level of the node itself and additional dots are going up the parent hierarchy. e.g. **${..foo}** points to the **foo** sibling of the parent of the current node.

In [17]:
conf = OmegaConf.load('../source/config_interpolation.yaml')
print(OmegaConf.to_yaml(conf))

server:
  host: localhost
  port: 80
client:
  url: http://${server.host}:${server.port}/
  server_port: ${server.port}
  description: Client of ${.url}



In [18]:
# Primitive interpolation types are inherited from the referenced value
print("conf.client.server_port: ", conf.client.server_port, type(conf.client.server_port).__name__)
# Composite interpolation types are always string
print("conf.client.url: ", conf.client.url, type(conf.client.url).__name__)

conf.client.server_port:  80 int
conf.client.url:  http://localhost:80/ str


`to_yaml()` will resolve interpolations if `resolve=True` is passed

In [19]:
print(OmegaConf.to_yaml(conf, resolve=True))

server:
  host: localhost
  port: 80
client:
  url: http://localhost:80/
  server_port: 80
  description: Client of http://localhost:80/



Interpolations may be nested, enabling more advanced behavior like dynamically selecting a sub-config:

In [20]:
cfg = OmegaConf.create(
    {
        "plans": {"A": "plan A", "B": "plan B"},
        "selected_plan": "A",
        "plan": "${plans.${selected_plan}}",
    }
)
print(f"Default: cfg.plan = {cfg.plan}")
cfg.selected_plan = "B"
print(f"After selecting plan B: cfg.plan = {cfg.plan}")

Default: cfg.plan = plan A
After selecting plan B: cfg.plan = plan B


Interpolated nodes can be any node in the config, not just leaf nodes:

In [21]:
cfg = OmegaConf.create(
    {
        "john": {"height": 180, "weight": 75},
        "player": "${john}",
    }
)
(cfg.player.height, cfg.player.weight)

(180, 75)

## Environment variable interpolation

Environment variable interpolation is also supported.
An environment variable is always returned as a string.

In [22]:
# Let's set up the environment first (only needed for this demonstration)
import os
os.environ['USER'] = 'omry'

Here is an example config file interpolates with the USER environment variable:

In [23]:
conf = OmegaConf.load('../source/env_interpolation.yaml')
print(OmegaConf.to_yaml(conf))

user:
  name: ${oc.env:USER}
  home: /home/${oc.env:USER}



In [24]:
conf = OmegaConf.load('../source/env_interpolation.yaml')
print(OmegaConf.to_yaml(conf, resolve=True))

user:
  name: omry
  home: /home/omry



You can specify a default value to use in case the environment variable is not defined.
This default value can be a string or ``null`` (representing Python ``None``). Passing a default with a different type will result in an error.

The following example sets default database passwords when ``DB_PASSWORD`` is not defined:

In [25]:
os.environ.pop('DB_PASSWORD', None)  # ensure env variable does not exist
cfg = OmegaConf.create(
    {
        "database": {
            "password1": "${oc.env:DB_PASSWORD,abc123}",  # the string 'abc123'
            "password2": "${oc.env:DB_PASSWORD,'12345'}",  # the string '12345'
        },
    }
)
print(repr(cfg.database.password1))
print(repr(cfg.database.password2))

'abc123'
'12345'


## Decoding strings with interpolations

You can automatically convert a string to its corresponding type (e.g., bool, int, float, dict, list) using `oc.decode` (which can even resolve interpolations).
This resolver also accepts ``None`` as input, in which case it returns ``None``.

This can be useful for instance to parse environment variables:

In [26]:
cfg = OmegaConf.create(
    {
        "database": {
            "port": '${oc.decode:${oc.env:DB_PORT}}',
            "nodes": '${oc.decode:${oc.env:DB_NODES,null}}',
            "timeout": '${oc.decode:${oc.env:DB_TIMEOUT,null}}',
        }
    }
)

os.environ["DB_PORT"] = "3308"  # integer
os.environ["DB_NODES"] = "[host1, host2, host3]"  # list
os.environ.pop("DB_TIMEOUT", None)  # unset variable

print("port (int):", repr(cfg.database.port))
print("nodes (list):", repr(cfg.database.nodes))
print("timeout (missing variable):", repr(cfg.database.timeout))

os.environ["DB_TIMEOUT"] = "${.port}"
print("timeout (interpolation):", repr(cfg.database.timeout))

port (int): 3308
nodes (list): ['host1', 'host2', 'host3']
timeout (missing variable): None
timeout (interpolation): 3308


## Custom interpolations

You can add additional interpolation types using custom resolvers.
The example below creates a resolver that adds 10 to the given value.

In [27]:
OmegaConf.register_new_resolver("plus_10", lambda x: x + 10)
conf = OmegaConf.create({'key': '${plus_10:990}'})
conf.key

1000

You can take advantage of nested interpolations to perform custom operations over variables:

In [28]:
OmegaConf.register_new_resolver("plus", lambda x, y: x + y)
conf = OmegaConf.create({"a": 1, "b": 2, "a_plus_b": "${plus:${a},${b}}"})
conf.a_plus_b

3

By default a custom resolver is called on every access, but it is possible to cache its output
by registering it with ``use_cache=True``.
This may be useful either for performance reasons or to ensure the same value is always returned.
Note that the cache is based on the string literals representing the resolver's inputs, and not
the inputs themselves:

In [29]:
import random
random.seed(1234)

OmegaConf.register_new_resolver("cached", random.randint, use_cache=True)
OmegaConf.register_new_resolver("uncached", random.randint)

cfg = OmegaConf.create(
    {
        "cached_1": "${cached:0,10000}",
        "uncached": "${uncached:0,10000}",
        "cached_2": "${cached:0,${uncached}}",
    }
)

# same value on repeated access thanks to the cache
print("With cache:", cfg.cached_1, "==", cfg.cached_1)

# not the same since the cache is disabled
print("Without cache:", cfg.uncached, "!=", cfg.uncached)

# same value even if `uncached` changes, because the cache is based
# on the string literal "${uncached}" that remains the same
print("With cache (interpolation):", cfg.cached_2, "==", cfg.cached_2)

With cache: 7220 == 7220
Without cache: 1914 != 122
With cache (interpolation): 1192 == 1192


# Merging configurations
Merging configurations enables the creation of reusable configuration files for each logical component instead of a single config file for each variation of your task.

Machine learning experiment example:
```python
conf = OmegaConf.merge(base_cfg, model_cfg, optimizer_cfg, dataset_cfg)
```

Web server configuration example:

```python
conf = OmegaConf.merge(server_cfg, plugin1_cfg, site1_cfg, site2_cfg)
```

The following example creates two configs from files, and one from the cli. It then combines them into a single object. Note how the port changes to 82, and how the users lists are combined.

In [30]:
base_conf = OmegaConf.load('../source/example2.yaml')
print(OmegaConf.to_yaml(base_conf))

server:
  port: 80
users:
- user1
- user2



In [31]:
second_conf = OmegaConf.load('../source/example3.yaml')
print(OmegaConf.to_yaml(second_conf))

log:
  file: log.txt



In [32]:
from omegaconf import OmegaConf
import sys

# Merge configs:
conf = OmegaConf.merge(base_conf, second_conf)

# Simulate command line arguments
sys.argv = ['program.py', 'server.port=82']
# Merge with cli arguments
conf.merge_with_cli()
print(OmegaConf.to_yaml(conf))

server:
  port: 82
users:
- user1
- user2
log:
  file: log.txt

